In [1]:
import itertools
from parlay_system import *
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


In [2]:
barys = MoneyLine(event="barys", bet_amount=100, odds=163)
ska = MoneyLine(event="ska", bet_amount=100, odds=-200)

neftekhimik = MoneyLine(event="neftekhimik", bet_amount=100, odds=163)
akbars = MoneyLine(event="akbars", bet_amount=100, odds=-200)

# 
# hawaii = MoneyLine(event="hawaii", bet_amount=100, odds=230)
# wash = MoneyLine(event="wash", bet_amount=100, odds=-295)

binaries = [
    [barys, ska],
    [neftekhimik, akbars],
]

results = [
    [0, 1], #[ska, barys],
    [0, 1], #[neftekhimik, akbars],
]


override_arr = [0, 2, 99]


def create_dicts(in_binaries, in_results):
    ml_lookup = {}
    ml_win_lookup = {}
    id = 0
    for bi in range(len(in_binaries)):
        binary = in_binaries[bi]
        for ml_i in range(len(binary)):
            ml = binary[ml_i]
            ml.set_index(id) 
            is_winner = in_results[bi][ml_i]
            ml_lookup[id] = ml.event
            ml_win_lookup[id] = is_winner
            id += 1
    
    return ml_lookup, ml_win_lookup

ml_lookup, ml_outcome_lookup = create_dicts(binaries, results)

In [3]:
def findsubsets(s, n):
    return list(itertools.combinations(s, n))

TOURNEY_SIZE = len(binaries)
SELECT_NUM = 2
s = set()
n = SELECT_NUM

for i in range(TOURNEY_SIZE):
    s.add(i)

all_subsets = findsubsets(s, n)

In [4]:
len(all_subsets)

1

In [5]:
csv = []
df = pd.DataFrame()

for i in range(len(all_subsets)):
    current_subset = all_subsets[i]
    parlay_binaries = []
    for i in range(len(current_subset)):
        index = current_subset[i]
        parlay_binaries.append(binaries[index])

    ps = ParlaySystem(binaries=parlay_binaries,
                    target_profit=0.15,
                    bounds=(0.005, 30),
                    binary_index_arr=list(current_subset),
                    binary_results_arr=results,
                    index_to_ml=ml_lookup,
                    index_to_outcome=ml_outcome_lookup,
                    override_arr=override_arr
                    )
    
    csv_res, raw_df = ps.slsqp_solver()
    df = pd.concat([df, raw_df])
    csv.append(csv_res)

     fun: -0.036097633893489196
     jac: array([-0.729225,  0.01375 ,  0.01375 ,  0.4375  ])
 message: 'Positive directional derivative for linesearch'
    nfev: 44
     nit: 8
    njev: 4
  status: 8
 success: False
       x: array([0.05580443, 0.08759463, 0.08759463, 0.005     ])
slsqp_solver: 
               event index[]  result  event_status    odds    bet    mult  payout  profit
3         ska_akbars  [1, 3]  [1, 1]          True  125.00  0.005  2.2500  0.0113  -0.225
0  barys_neftekhimik  [0, 2]  [0, 0]         False  591.69  0.056  6.9169  0.3860   0.150
1       barys_akbars  [0, 3]  [0, 1]         False  294.50  0.088  3.9450  0.3456   0.110
2    ska_neftekhimik  [1, 2]  [1, 0]         False  294.50  0.088  3.9450  0.3456   0.110
total_bet   :  0.24
mean        :  0.03625
max         :  0.15
min         :  -0.225
std         :  0.1751844266290053
             odds       bet      mult    payout    profit
count    4.000000  4.000000  4.000000  4.000000  4.000000
mean   326.42250

In [32]:
a = MoneyLine(event="neg", bet_amount=0.20355555555555556, odds=125.00)
a.print_stats()

  event  bet_amount   odds  multiplier  payout
0   neg    0.203556  125.0        2.25   0.458


In [28]:
b = MoneyLine(event="neg", bet_amount=0.056, odds=591.69)
b.print_stats()

  event  bet_amount    odds  multiplier    payout
0   neg       0.056  591.69      6.9169  0.387346


In [29]:
c = MoneyLine(event="neg", bet_amount=0.088, odds=294.50)
c.print_stats()

  event  bet_amount   odds  multiplier   payout
0   neg       0.088  294.5       3.945  0.34716


In [30]:
d = MoneyLine(event="neg", bet_amount=0.088, odds=294.50)
d.print_stats()

  event  bet_amount   odds  multiplier   payout
0   neg       0.088  294.5       3.945  0.34716


In [31]:
a.bet_amount + b.bet_amount + c.bet_amount + d.bet_amount

0.33555555555555555

In [7]:
df.describe()

,odds,bet,mult,payout,profit
count,4.000000,4.000000,4.000000,4.000000,4.000000
mean,326.422500,0.059250,4.264225,0.272125,0.036250
std,194.058378,0.039187,1.940584,0.174923,0.175184
min,125.000000,0.005000,2.250000,0.011300,-0.225000
25%,252.125000,0.043250,3.521250,0.262025,0.026250
50%,294.500000,0.072000,3.945000,0.345600,0.110000
75%,368.797500,0.088000,4.687975,0.355700,0.120000
max,591.690000,0.088000,6.916900,0.386000,0.150000


In [8]:
negative_df = df.loc[df.profit < 0]
negative_df = negative_df.sort_values(by=['event_status', 'profit'], ascending=[False, True])
negative_df.describe()

,odds,bet,mult,payout,profit
count,1.0,1.000,1.00,1.0000,1.000
mean,125.0,0.005,2.25,0.0113,-0.225
std,NaN,NaN,NaN,NaN,NaN
min,125.0,0.005,2.25,0.0113,-0.225
25%,125.0,0.005,2.25,0.0113,-0.225
50%,125.0,0.005,2.25,0.0113,-0.225
75%,125.0,0.005,2.25,0.0113,-0.225
max,125.0,0.005,2.25,0.0113,-0.225


In [9]:
positive_df = df.loc[df.profit >= 0]
positive_df = positive_df.sort_values(by=['event_status', 'profit'], ascending=[False, False])
positive_df.describe()

,odds,bet,mult,payout,profit
count,3.000000,3.000000,3.000000,3.000000,3.000000
mean,393.563333,0.077333,4.935633,0.359067,0.123333
std,171.582727,0.018475,1.715827,0.023325,0.023094
min,294.500000,0.056000,3.945000,0.345600,0.110000
25%,294.500000,0.072000,3.945000,0.345600,0.110000
50%,294.500000,0.088000,3.945000,0.345600,0.110000
75%,443.095000,0.088000,5.430950,0.365800,0.130000
max,591.690000,0.088000,6.916900,0.386000,0.150000


In [10]:
negative_df.head(200)

,event,index[],result,event_status,odds,bet,mult,payout,profit
3,ska_akbars,"[1, 3]","[1, 1]",True,125.0,0.005,2.25,0.0113,-0.225


In [11]:
positive_df.head(200)

,event,index[],result,event_status,odds,bet,mult,payout,profit
0,barys_neftekhimik,"[0, 2]","[0, 0]",False,591.69,0.056,6.9169,0.3860,0.15
1,barys_akbars,"[0, 3]","[0, 1]",False,294.50,0.088,3.9450,0.3456,0.11
2,ska_neftekhimik,"[1, 2]","[1, 0]",False,294.50,0.088,3.9450,0.3456,0.11


In [12]:
positive_df.groupby('event_status').count()

,event,index[],result,odds,bet,mult,payout,profit
event_status,,,,,,,,
False,3,3,3,3,3,3,3,3


In [13]:
negative_df.groupby('event_status').count()

,event,index[],result,odds,bet,mult,payout,profit
event_status,,,,,,,,
True,1,1,1,1,1,1,1,1


In [14]:
neg = negative_df.reset_index(drop=True).style.applymap(color_negative_red)
neg

,event,index[],result,event_status,odds,bet,mult,payout,profit
0,ska_akbars,"['1', '3']","[1, 1]",True,125,0.005,2.25,0.0113,-0.225


In [15]:
pos = positive_df.reset_index(drop=True).style.applymap(color_positive_green)
pos

,event,index[],result,event_status,odds,bet,mult,payout,profit
0,barys_neftekhimik,"['0', '2']","[0, 0]",False,591.69,0.056,6.9169,0.386,0.15
1,barys_akbars,"['0', '3']","[0, 1]",False,294.5,0.088,3.945,0.3456,0.11
2,ska_neftekhimik,"['1', '2']","[1, 0]",False,294.5,0.088,3.945,0.3456,0.11


In [16]:
sf49ers = MoneyLine(event="parlay", bet_amount=100, odds=1002)
sf49ers.print_stats()

    event  bet_amount  odds  multiplier  payout
0  parlay         100  1002       11.02  1102.0


In [17]:
pos = MoneyLine(event="pos", bet_amount=100, odds=-1020)
pos.print_stats()

  event  bet_amount  odds  multiplier      payout
0   pos         100 -1020    1.098039  109.803922


In [18]:
neg = MoneyLine(event="neg", bet_amount=1.0, odds=-305)
neg.print_stats()

  event  bet_amount  odds  multiplier    payout
0   neg         1.0  -305    1.327869  1.327869


In [19]:
neg = MoneyLine(event="neg", bet_amount=1.0, odds=-285)
neg.print_stats()

  event  bet_amount  odds  multiplier    payout
0   neg         1.0  -285    1.350877  1.350877
